# To Check if Spark is active

In [1]:
import pyspark
sc

<SparkContext master=local[*] appName=PySparkShell>

# Converting the data into Spark DataFrames

In [ ]:
movie_df=spark.read.csv('movie.csv',inferSchema=True,header=True)
link_df=spark.read.csv('link.csv',inferSchema=True,header=True)
rating_df=spark.read.csv('rating.csv',inferSchema=True,header=True)
tag_df=spark.read.csv('tag.csv',inferSchema=True,header=True)

In [ ]:
# Movies dataframe
movie_df.show()

In [ ]:
# Link dataframe
link_df.show()

In [ ]:
#Tags Dataframe
tag_df.show()

In [ ]:
#Ratings Dataframe
rating_df.show()

# Extracting the year and the genres for all movies

In [ ]:
movie_df.select("title","genres").show()

In [ ]:
# Testing and normally extracting the year
movie_df_title_yr = movie_df.select("title").collect()
final_yr = []
for i in movie_df_title_yr:
    final_yr.append(i[0][-5:-1])
final_yr[0:10]


In [ ]:
#Extracting the year in the title column and showing it with genre

from pyspark.sql.functions import regexp_replace,col
from pyspark.sql.functions import udf
import re

def years(s):
    return s[-5:-1]

yr_udf = udf(years)

movie_df.withColumn("title", yr_udf("title")).show()

# RDD - What are the top 10 most viewed movies?

In [ ]:

## set up SparkSession
import pyspark
from pyspark.sql import SparkSession

MAX_MEMORY = "15g"

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("RDD with movielens") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

# Get ratings csv
df = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
                               load("rating.csv",header=True)

df.show(5)
df.printSchema()

In [2]:
# Better RDD method
df = sc.textFile("rating.csv")
df.take(5)

['"userId","movieId","rating","timestamp"',
 '1,2,3.5,2005-04-02 23:53:47',
 '1,29,3.5,2005-04-02 23:31:16',
 '1,32,3.5,2005-04-02 23:33:39',
 '1,47,3.5,2005-04-02 23:32:07']

In [3]:
# Movie pairs
df = df.map(lambda x: (x,1))

In [4]:
df.take(4)

[('"userId","movieId","rating","timestamp"', 1),
 ('1,2,3.5,2005-04-02 23:53:47', 1),
 ('1,29,3.5,2005-04-02 23:31:16', 1),
 ('1,32,3.5,2005-04-02 23:33:39', 1)]

In [5]:
from pyspark import SparkContext
SparkContext.setSystemProperty('spark.driver.memory', '15g')
SparkContext.setSystemProperty('spark.executor.memory', '15g')

In [6]:
import numpy as np
arr = np.array(df.collect())
arr

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 2.0 failed 1 times, most recent failure: Lost task 15.0 in stage 2.0 (TID 17, ARCEUS, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:232)
	at org.apache.spark.scheduler.DirectTaskResult.$anonfun$writeExternal$1(Ta

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 59146)
Traceback (most recent call last):
  File "c:\users\my1\appdata\local\programs\python\python36\lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\users\my1\appdata\local\programs\python\python36\lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "c:\users\my1\appdata\local\programs\python\python36\lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\users\my1\appdata\local\programs\python\python36\lib\socketserver.py", line 721, in __init__
    self.handle()
  File "C:\Users\My1\Desktop\BigData\spark-3.0.1-bin-hadoop2.7\python\pyspark\accumulators.py", line 268, in handle
    poll(accum_updates)
  File "C:\Users\My1\Desktop\BigData\spark-3.0.1-bin-hadoop2.7\python\pyspark\accumulator

In [ ]:
# Movie Count
df = df.reduceByKey(lambda x,y : x+y)

In [ ]:
df.take(2)

In [ ]:
# Movies sorted
df = df.sortBy((lambda x : x[2]),ascending = False,numPartitions = None)

In [ ]:
# Final list 
mv_top10List=list(df.take(10))

In [ ]:
mv_top10List